**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [182]:
# Write your code here
import numpy as np
from types import SimpleNamespace
from scipy import optimize

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [183]:
par = SimpleNamespace()

# firms
par.A = 1.0
par.gamma = 0.5

# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

# government
par.tau = 0.0
par.T = 0.0

# Question 3
par.kappa = 0.1

# Numeraire
w = 1

In [184]:
p1_range = np.linspace(0.1, 2, 10)
p2_range = np.linspace(0.1, 2, 10)

def profits(p, w, par):
    
    # a. solve
    y = lambda l: par.A*l**par.gamma
    obj = lambda l: -(p*y(l) - w*l)
    x0 = [0.5]
    res = optimize.minimize(obj,x0, bounds=((0,None),), method='L-BFGS-B')

    # b. output
    l_star = (res.x[0])**(1/(1-par.gamma))
    y_star = par.A*y(l_star)**(par.gamma)
    Pi = (1-par.gamma)/(par.gamma)*w*(p*par.A*par.gamma/w)**(1/(1-par.gamma))

    return Pi, y_star, l_star

**Small test**

In [185]:
w = 1
for p in p1_range:
    pi, y, l = profits(p, w, par)
    print(f'p = {p:.2f}, pi = {pi:.2f}, y = {y:.2f}, l = {l:.2f}')

p = 0.10, pi = 0.00, y = 0.05, l = 0.00
p = 0.31, pi = 0.02, y = 0.16, l = 0.00
p = 0.52, pi = 0.07, y = 0.26, l = 0.00
p = 0.73, pi = 0.13, y = 0.37, l = 0.02
p = 0.94, pi = 0.22, y = 0.47, l = 0.05
p = 1.16, pi = 0.33, y = 0.58, l = 0.11
p = 1.37, pi = 0.47, y = 0.68, l = 0.22
p = 1.58, pi = 0.62, y = 0.79, l = 0.39
p = 1.79, pi = 0.80, y = 0.89, l = 0.64
p = 2.00, pi = 1.00, y = 1.00, l = 1.00


In [191]:
def utility(c1,c2,l,par):
    return np.log(c1**par.alpha * c2**(1-par.alpha)) - par.nu*l**(1+par.epsilon)/(1+par.epsilon)

def consumer(p,w,pi,par):
    # a. solve
    obj = lambda l: -utility(par.alpha*(w*l+par.T+pi)/p,(1-par.alpha)*(w*l+par.T+pi)/(p+par.tau),l,par)
    res = optimize.minimize_scalar(obj, bounds=(0,1), method='bounded')

    # b. output
    l_star = res.x
    c1_star = par.alpha*(w*l_star+par.T+pi)/p
    c2_star = (1-par.alpha)*(w*l_star+par.T+pi)/(p+par.tau)

    return c1_star, c2_star, l_star

**Small test**

In [193]:
w = 1
pi = 0.5
for p in p1_range:
    c1,c2,l = consumer(p,w,pi,par)
    print(f'p = {p:.2f}, c1 = {c1:.2f}, c2 = {c2:.2f}, l = {l:.2f}')

p = 0.10, c1 = 4.07, c2 = 9.51, l = 0.86
p = 0.31, c1 = 1.31, c2 = 3.06, l = 0.86
p = 0.52, c1 = 0.78, c2 = 1.82, l = 0.86
p = 0.73, c1 = 0.56, c2 = 1.30, l = 0.86
p = 0.94, c1 = 0.43, c2 = 1.01, l = 0.86
p = 1.16, c1 = 0.35, c2 = 0.82, l = 0.86
p = 1.37, c1 = 0.30, c2 = 0.70, l = 0.86
p = 1.58, c1 = 0.26, c2 = 0.60, l = 0.86
p = 1.79, c1 = 0.23, c2 = 0.53, l = 0.86
p = 2.00, c1 = 0.20, c2 = 0.48, l = 0.86


**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [275]:
def equilibrium(p1,p2,w,par):
    # a. labor demand
    pi1, y1, l1 = profits(p1,w,par)
    pi2, y2, l2 = profits(p2,w,par)
    L = l1 + l2

    # b. labor supply
    c1, c2, L = consumer(p1,w,pi1,par)
    C = c1 + c2

    # c. market clearing
    Labor_market_clearing = L - l1 + l2
    Goods_market_clearing_1 = y1 - c1
    Goods_market_clearing_2 = y2 - c2

    return Labor_market_clearing, Goods_market_clearing_1, Goods_market_clearing_2

In [279]:
w = 1
for p1 in p1_range:
    for p2 in p2_range:
        Labor_market_clearing, Goods_market_clearing_1, Goods_market_clearing_2 = equilibrium(p1,p2,w,par)
        print(f'p1 = {p1:.2f}, p2 = {p2:.2f} ,LMC: {Labor_market_clearing:.2f}, GM1C: {Goods_market_clearing_1:.2f} , GM2C: {Goods_market_clearing_2:.2f}')

p1 = 0.10, p2 = 0.10 ,LMC: 1.00, GM1C: -2.96 , GM2C: -6.96
p1 = 0.10, p2 = 0.31 ,LMC: 1.00, GM1C: -2.96 , GM2C: -6.86
p1 = 0.10, p2 = 0.52 ,LMC: 1.00, GM1C: -2.96 , GM2C: -6.75
p1 = 0.10, p2 = 0.73 ,LMC: 1.02, GM1C: -2.96 , GM2C: -6.65
p1 = 0.10, p2 = 0.94 ,LMC: 1.05, GM1C: -2.96 , GM2C: -6.54
p1 = 0.10, p2 = 1.16 ,LMC: 1.11, GM1C: -2.96 , GM2C: -6.43
p1 = 0.10, p2 = 1.37 ,LMC: 1.22, GM1C: -2.96 , GM2C: -6.33
p1 = 0.10, p2 = 1.58 ,LMC: 1.39, GM1C: -2.96 , GM2C: -6.22
p1 = 0.10, p2 = 1.79 ,LMC: 1.64, GM1C: -2.96 , GM2C: -6.12
p1 = 0.10, p2 = 2.00 ,LMC: 2.00, GM1C: -2.96 , GM2C: -6.01
p1 = 0.31, p2 = 0.10 ,LMC: 0.99, GM1C: -0.82 , GM2C: -2.24
p1 = 0.31, p2 = 0.31 ,LMC: 0.99, GM1C: -0.82 , GM2C: -2.13
p1 = 0.31, p2 = 0.52 ,LMC: 1.00, GM1C: -0.82 , GM2C: -2.03
p1 = 0.31, p2 = 0.73 ,LMC: 1.01, GM1C: -0.82 , GM2C: -1.92
p1 = 0.31, p2 = 0.94 ,LMC: 1.04, GM1C: -0.82 , GM2C: -1.81
p1 = 0.31, p2 = 1.16 ,LMC: 1.10, GM1C: -0.82 , GM2C: -1.71
p1 = 0.31, p2 = 1.37 ,LMC: 1.21, GM1C: -0.82 , GM2C: -1.

Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [ ]:
# write your answer here
# Calculate the social welfare function for each combination of tau and T
swf_values = []
for tau in tau_range:
    for T in T_range:
        # Calculate the SWF for the current values of tau and T
        swf = calculate_swf(tau, T)
        
        # Calculate the implied value of T
        implied_T = (1 - tau) * T
        
        # Calculate the SWF with the social cost of carbon
        swf_with_cost = swf - par.kappa * implied_T
        
        # Append the SWF value to the list
        swf_values.append((swf_with_cost, tau, implied_T))
        
# Find the maximum SWF value and corresponding values of tau and implied T
max_swf_with_cost, opt_tau, opt_implied_T = max(swf_values)

# Print the optimal values of tau and implied T
print("Optimal tau:", opt_tau)
print("Implied optimal T:", opt_implied_T)

Optimal tau: 0.1
Implied optimal T: 0.09000000000000001


## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [ ]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 10000

par.F = np.arange(1,par.N+1)
par.sigma = 2

par.v = np.array([1,2,3])
par.c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [ ]:
# write your answer here

Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [ ]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [ ]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [ ]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [ ]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [ ]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [ ]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [ ]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [ ]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [ ]:
# write your answer here